In [4]:
!pip install earthengine-api


In [5]:
!earthengine-api --version

'earthengine-api' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
!earthengine authenticate
!pip install google-auth-oauthlib google-auth-httplib2

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=http%3A%2F%2Flocalhost%3A8085&response_type=code&code_challenge=29YPjiJU_S04h6bQZ28otf7qdx4ToPJgcz3bBhQ0CqU&code_challenge_method=S256

Waiting for successful authorization from web browser ...

Successfully saved authorization token.


In [16]:
final_data =[]

In [8]:
import ee
import google.auth
from google.oauth2 import service_account

#credentials = service_account.Credentials.from_service_account_file('/content/drive/MyDrive/earthengine-credentials.json')
#ee.Initialize(credentials)
ee.Authenticate()
ee.Initialize(project='propane-passkey-421619')


    # Process and reduce the datasets
    # Example: Calculate the mean temperature and sum of flood occurrences
    # mean_temperature = terraclimate.select('tmmx').mean()  # Mean of max temp across time
    # flood_count = flood_dataset.select('flooded').sum()   # Sum of flood occurrences across time

    # # Clip the data to the area of interest
    # mean_temperature_clip = mean_temperature.clip(area_of_interest)
    # flood_count_clip = flood_count.clip(area_of_interest)

    # # Export the result to Google Drive
    # export_to_drive(mean_temperature_clip, "Mean_Temperature")
    # export_to_drive(flood_count_clip, "Flood_Count")


# def export_to_drive(image, description):
#     """Helper function to export Image to Google Drive"""
#     task = ee.batch.Export.image.toDrive(
#         image=image,
#         description=description,
#         folder='EarthEngineData',
#         scale=30,
#         region=image.geometry().getInfo()['coordinates'],
#         fileFormat='GeoTIFF'
#     )
#     task.start()




In [17]:
def nextMonth(date):
    yy, mm, dd = date.split('-')
    if int(mm) == 12:
        mm = '01'
        yy = str(int(yy) + 1)
        print('year now is {yy} ')
    else:
        mm = str(int(mm) + 1).zfill(2)
    return f'{yy}-{mm}-{dd}'



In [18]:
area_of_interest = ee.Geometry.Rectangle([89, 22, 91, 25])  # Example: Longitude and Latitude


start_date = '2000-03-01'
end_date = '2000-04-01'
while not end_date=='2018-12-01':


# Load TerraClimate dataset
  terraclimate = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE").filterDate(start_date, end_date).filterBounds(area_of_interest)

    # Load Global Flood Database
  flood_dataset = ee.ImageCollection("GLOBAL_FLOOD_DB/MODIS_EVENTS/V1").filterDate(start_date, end_date).filterBounds(area_of_interest)
  # try:
  #   flood_dataset.mean().select('flooded').getInfo()
  # except:
  #   nextMonth(start_date)
  #   nextMonth(end_date)
  #   continue

  yi=22

  ye=22.5
  for i in range(6):
    xi=89
    xe=89.5
    for j in range(4):
      region_of_interest = ee.Geometry.Rectangle([xi,yi ,xe ,ye])

      mean_terra = terraclimate.mean().reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=region_of_interest,
      scale=4166.67,  # Match this scale to the data's resolution, which is about 4km for TerraClimate
      maxPixels=1e9
      )
        # Applying reduceRegion to the resampled flood dataset
      mean_flood = flood_dataset.mean().reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=region_of_interest,
      scale=4166.67,  # Use the same scale as TerraClimate for consistency
      maxPixels=1e9)
        # Retrieve the information
      mean_terra_info = mean_terra.getInfo()
      mean_flood_info = mean_flood.getInfo()
      combined_row = {**mean_terra_info, **mean_flood_info}
      final_data.append(combined_row)
      # print(combined_row)
      xi=xi+0.5
      xe=xe+0.5
    yi=yi+0.5
    ye=ye+0.5

  # print("Mean terra:", mean_terra_info)
  # print("Mean Flooded Area:", mean_flood_info)
  # print("combined row:", combined_row)


  start_date =nextMonth(start_date)
  end_date = nextMonth(end_date)



year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 
year now is {yy} 


In [19]:
final_data

[{'aet': 557.8495349501964,
  'def': 827.5079926999268,
  'pdsi': 115.0270674377184,
  'pet': 1385.3854306383155,
  'pr': 17.174827942566882,
  'ro': 1,
  'soil': 788.2360210206901,
  'srad': 2395.887552496755,
  'swe': 0,
  'tmmn': 209.52637481035168,
  'tmmx': 314.274412366147,
  'vap': 2482.162140768265,
  'vpd': 107.83482486422298,
  'vs': 205.54343763055456},
 {'aet': 598.0685795954266,
  'def': 784.9330255057166,
  'pdsi': 101.21319261213723,
  'pet': 1382.991622691293,
  'pr': 25.58276165347406,
  'ro': 1.0947669305189096,
  'soil': 724.8822559366752,
  'srad': 2394.1142040457344,
  'swe': 0,
  'tmmn': 209.00241864555846,
  'tmmx': 314.2810026385224,
  'vap': 2481.5193491644677,
  'vpd': 107.57678100263851,
  'vs': 204.5824098504837},
 {'aet': 613.9781504846901,
  'def': 740.5720000879261,
  'pdsi': 97.97058888180605,
  'pet': 1354.5595146506066,
  'pr': 33.18134658079265,
  'ro': 1.7769766777306404,
  'soil': 636.9090849141629,
  'srad': 2393.6741916338783,
  'swe': 0,
  'tmmn'

In [20]:
import pandas as pd

# Create a list


# Convert the list to a DataFrame
df = pd.DataFrame(final_data)

# Save the DataFrame to a CSV file
df.to_csv('final_data.csv', index=False)

In [26]:
df_cleaned = df.dropna(subset=['flooded'])

In [28]:
df_cleaned.head()

,aet,def,pdsi,pet,pr,ro,soil,srad,swe,tmmn,tmmx,vap,vpd,vs,clear_perc,clear_views,duration,flooded,jrc_perm_water
144,1080.216715,0.0,-4.013017,1080.216715,324.614679,216.552827,2754.305680,1847.592427,0,250.609204,313.195475,3238.362893,65.961960,213.398140,0.988691,35.039689,0.247224,0.192022,0.214140
145,1082.170163,0.0,69.886961,1082.170163,342.510224,234.318734,2623.587687,1873.788083,0,248.978496,311.516161,3218.678386,64.203496,206.329200,0.981394,34.733619,0.002942,0.000718,0.106178
146,1084.916625,0.0,88.613457,1084.916625,364.659530,256.188513,2434.356494,1884.604906,0,249.153100,310.544172,3203.735036,64.524652,201.120920,0.973083,34.474746,0.010555,0.000409,0.035460
147,1077.275261,0.0,-27.817701,1077.275261,307.361285,199.669663,2464.823924,1877.217848,0,245.215202,311.201254,3179.765557,64.329312,195.577345,0.977747,34.589039,0.030054,0.020908,0.004807
148,1076.180991,0.0,121.580639,1076.180991,324.515218,216.905131,2282.000176,1841.876959,0,250.180200,317.387518,3239.413998,70.989693,172.126514,0.983067,34.900473,0.329531,0.262952,0.066623


In [30]:
df_cleaned.to_csv('cleaned_final_data.csv', index=False)

In [34]:
!pip3 install -U pip
!pip3 install -U setuptools wheel

!pip3 install autogluon

In [36]:
from autogluon.tabular import TabularDataset, TabularPredictor

AttributeError: module 'numpy' has no attribute '_no_nep50_warning'

In [ ]:
flood_dataset.mean().select('flooded').getInfo()

EEException: Image.select: Band pattern 'flooded' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands

In [ ]:
terraclimate_projection = terraclimate.first().projection()

flood_resampled = flood_dataset.mean().reproject(
    crs=terraclimate_projection.crs(),
    scale=terraclimate_projection.nominalScale()
)

In [ ]:
print(terraclimate_projection.crs())

ee.String({
  "functionInvocationValue": {
    "functionName": "Projection.crs",
    "arguments": {
      "projection": {
        "functionInvocationValue": {
          "functionName": "Image.projection",
          "arguments": {
            "image": {
              "functionInvocationValue": {
                "functionName": "Collection.first",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.filter",
                      "arguments": {
                        "collection": {
                          "functionInvocationValue": {
                            "functionName": "Collection.filter",
                            "arguments": {
                              "collection": {
                                "functionInvocationValue": {
                                  "functionName": "ImageCollection.load",
                                  "arguments": {
         

In [ ]:
flood_resampled.select('duration')

In [ ]:
# Define the region of interest as a rectangle (Example: Bounds for a specific area)
region_of_interest = ee.Geometry.Rectangle([89, 22, 89.5, 22.5])


In [ ]:
flood_resampled.select('flooded')

In [ ]:
# mean_flood_re = flood_resampled.reduceRegion(
#     reducer=ee.Reducer.mean(),
#     geometry=region_of_interest,
#     scale=4166.67,  # Use the same scale as TerraClimate for consistency
#     maxPixels=1e9,
#     crs = 'EPSG:4326'
# )

# Retrieve the information


In [ ]:

mean_terra = terraclimate.mean().reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=region_of_interest,
    scale=4166.67,  # Match this scale to the data's resolution, which is about 4km for TerraClimate
    maxPixels=1e9
    # crs= terraclimate_projection.crs()
)

# Applying reduceRegion to the resampled flood dataset
mean_flood = flood_dataset.mean().reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=region_of_interest,
    scale=4166.67,  # Use the same scale as TerraClimate for consistency
    maxPixels=1e9
)

# Retrieve the information
mean_terra_info = mean_terra.getInfo()
mean_flood_info = mean_flood.getInfo()
combined_row = {**mean_terra_info, **mean_flood_info}
final_data.append(combined_row)

print("Mean terra:", mean_terra_info)
print("Mean Flooded Area:", mean_flood_info)
print("combined row:", combined_row)


Mean terra: {'aet': 557.8495349501964, 'def': 827.5079926999268, 'pdsi': 115.0270674377184, 'pet': 1385.3854306383155, 'pr': 17.174827942566882, 'ro': 1, 'soil': 788.2360210206901, 'srad': 2395.887552496755, 'swe': 0, 'tmmn': 209.52637481035168, 'tmmx': 314.274412366147, 'vap': 2482.162140768265, 'vpd': 107.83482486422298, 'vs': 205.54343763055456}
Mean Flooded Area: {}
combined row: {'aet': 557.8495349501964, 'def': 827.5079926999268, 'pdsi': 115.0270674377184, 'pet': 1385.3854306383155, 'pr': 17.174827942566882, 'ro': 1, 'soil': 788.2360210206901, 'srad': 2395.887552496755, 'swe': 0, 'tmmn': 209.52637481035168, 'tmmx': 314.274412366147, 'vap': 2482.162140768265, 'vpd': 107.83482486422298, 'vs': 205.54343763055456}


In [ ]:
final_data

[{'aet': 557.8495349501964,
  'def': 827.5079926999268,
  'pdsi': 115.0270674377184,
  'pet': 1385.3854306383155,
  'pr': 17.174827942566882,
  'ro': 1,
  'soil': 788.2360210206901,
  'srad': 2395.887552496755,
  'swe': 0,
  'tmmn': 209.52637481035168,
  'tmmx': 314.274412366147,
  'vap': 2482.162140768265,
  'vpd': 107.83482486422298,
  'vs': 205.54343763055456},
 {'aet': 598.0685795954266,
  'def': 784.9330255057166,
  'pdsi': 101.21319261213723,
  'pet': 1382.991622691293,
  'pr': 25.58276165347406,
  'ro': 1.0947669305189096,
  'soil': 724.8822559366752,
  'srad': 2394.1142040457344,
  'swe': 0,
  'tmmn': 209.00241864555846,
  'tmmx': 314.2810026385224,
  'vap': 2481.5193491644677,
  'vpd': 107.57678100263851,
  'vs': 204.5824098504837},
 {'aet': 613.9781504846901,
  'def': 740.5720000879261,
  'pdsi': 97.97058888180605,
  'pet': 1354.5595146506066,
  'pr': 33.18134658079265,
  'ro': 1.7769766777306404,
  'soil': 636.9090849141629,
  'srad': 2393.6741916338783,
  'swe': 0,
  'tmmn'

In [ ]:
region = ee.Geometry.Rectangle([-123.1, 37.4, -121.8, 38.1])

# Reduce region. Here we take the mean, but other reducers like median, sum can also be used
mean_lst_in_region = flood_resampled.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=region,
    scale=1000,  # Set scale to match dataset resolution
    maxPixels=1e9
)

# Get the computed mean value
lst_mean_value = mean_lst_in_region.get('LST_Day_1km').getInfo()
print(f"Mean LST in the region is: {lst_mean_value}")

EEException: Projection: Parameter 'crs' is required.

In [ ]:
# precip = terraclimate.select('pr').mean()

# # Reduce region to get correlation
# correlation = precip.reduceRegion(
#     reducer=ee.Reducer.pearsonsCorrelation(),
#     geometry=area_of_interest,
#     scale=4166.67,
#     maxPixels=1e9
# )

# # Get the correlation result
# correlation_info = correlation.getInfo()
# print(correlation_info)

In [ ]:
# Aggregate the flood data to the coarser resolution of TerraClimate
flood_resampled = flood_dataset.mean().reproject({
    'crs': terraclimate.first().projection(),
    'scale': 4166.67  # approximately 4km
})


In [ ]:
# Example: Calculate the correlation between a TerraClimate variable and flood occurrences
# Let's assume 'precip' for TerraClimate and 'flooded' band for Flood data
precip = terraclimate.select('precip').mean()
flooded = flood_resampled.select('flooded')

# Use a Reducer to correlate the datasets
correlation = precip.reduceRegion({
    'reducer': ee.Reducer.pearsonsCorrelation(),
    'geometry': area_of_interest,
    'scale': 4166.67,
    'maxPixels': 1e9
})

# Get the correlation info
correlation_info = correlation.getInfo()
print(correlation_info)


TypeError: Reducer can only be used as a cast to Reducer. Found <class 'dict'>.

In [ ]:
mean_lst_in_region = flood_dataset.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=area_of_interest,
    scale=1000,  # Set scale to match dataset resolution
    maxPixels=1e9
)

# Get the computed mean value
lst_mean_value = mean_lst_in_region.get('LST_Day_1km').getInfo()
print(f"Mean LST in the region is: {lst_mean_value}")

AttributeError: 'ImageCollection' object has no attribute 'reduceRegion'